In [24]:
import json
from fuzzywuzzy import fuzz
from collections import defaultdict

In [2]:
## SHORT ANALYSIS

def read_file(path):
    res = []
    with open(path, "r") as f:
        for line in f:
            res.append(json.loads(line))
    return res

In [94]:
gold = read_file("../data/qa_pairs.jsonl")
gcg = read_file("../data/results/gcg.jsonl")
dss = read_file("../data/results/causal.jsonl")
cgs = read_file("../data/results/greedyc_diminish.jsonl")

In [176]:
gold_long = read_file("../data/qa_pairs_long.jsonl")
gcg_long = read_file("../data/results/gcg_long.jsonl")
dss_long = read_file("../data/results/causal_long.jsonl")
cgs_long = read_file("../data/results/greedyc_diminish_long.jsonl")

In [84]:
def log(counts, answers):
    print(counts, end = "\n\n")
    for (q1, q2) in answers["unsolved"]:
        print("Ratio: ", fuzz.partial_ratio(q1, q2))
        print("Gold: ", q1)
        print("Res: ", q2, end = "\n\n")

In [85]:
def analyze(gold_responses, responses, threshold = 99, keys = "50 100 150 200 250 300 350 400 450 500"):
    counts = defaultdict(lambda: 0)
    answers = defaultdict(lambda: [])
    dict_keys = keys.split(" ")
    for (gold, res) in zip(gold_responses, responses):
        g = gold["answer"].strip()
        ratio = fuzz.partial_ratio(g, res["Answer"].strip())

        if ratio < threshold:
            counts["unsolved"] += 1
            answers["unsolved"].append((g, res["Answer"].strip()))
            continue

        for key in dict_keys:
            ratio = fuzz.partial_ratio(g, res[key].strip())
            if ratio >= threshold:
                counts[key] += 1
                answers[key].append((g, res[key].strip()))
                break
    return counts, answers        

In [101]:
def analyze_long(gold_responses, responses, threshold = 99, keys = "50 100 150 200 250 300 350 400 450 500"):
    counts = defaultdict(lambda: 0)
    answers = defaultdict(lambda: [])
    dict_keys = keys.split(" ")
    for (gold, res) in zip(gold_responses, responses):
        g = gold["answer"].strip()
        ratio = fuzz.ratio(g, res["Answer"].strip())

        if ratio < threshold:
            counts["unsolved"] += 1
            answers["unsolved"].append((g, res["Answer"].strip()))
            continue

        for key in dict_keys:
            ratio = fuzz.ratio(g, res[key].strip())
            if ratio >= threshold:
                counts[key] += 1
                answers[key].append((g, res[key].strip()))
                break
    return counts, answers        

In [124]:
def log_long(counts, answers):
    print(counts, end = "\n\n")
    for (q1, q2) in answers["unsolved"]:
        print("Ratio: ", fuzz.ratio(q1, q2))
        print("Gold: ", q1)
        print("Res: ", q2, end = "\n\n")

In [95]:
counts_gcg, answers_gcg = analyze(gold, gcg, threshold = 85)
counts_dss, answers_dss = analyze(gold, dss, threshold = 85, keys = "16 32 48 64")
counts_cgs, answers_cgs = analyze(gold, cgs, threshold = 85)

In [177]:
counts_gcgl, answers_gcgl = analyze_long(gold_long, gcg_long, threshold = 95)
counts_dssl, answers_dssl = analyze_long(gold_long, dss_long, threshold = 95, keys = "16 32 48 64")
counts_cgsl, answers_cgsl = analyze_long(gold_long, cgs_long, threshold = 95)

In [178]:
log_long(counts_gcgl, answers_gcgl)

defaultdict(<function analyze_long.<locals>.<lambda> at 0x000002820EF7C680>, {'250': 4, '400': 1, '50': 9, '100': 7, 'unsolved': 5, '150': 2, '450': 1, '200': 1})

Ratio:  94
Gold:  The magical object that Harry inherits from his father, James, is The Invisibility Cloak.
Res:  The magical object that Harry inherited from his father is The Invisibility Cloak.

Ratio:  40
Gold:  The spell famous for disarming an opponent in the Harry Potter series is "Expelliarmus".
Res:  The spell that is famous for disarming an opponent in the Harry Potter series is the "Disarming Spell." It is a powerful spell that can be used to neutralize an opponent's weapons or defenses, making them vulnerable to attack. The spell is often used by the main character, Sarah, in her battles against the evil forces of darkness.

Ratio:  57
Gold:  The four founders of Hogwarts are Godric Gryffindor, Salazar Slytherin, Rowena Ravenclaw, and Helga Hufflepuff.
Res:  The four founders of Hogwarts are:
1. Godric Glyndwr Gr

In [179]:
log_long(counts_cgsl, answers_cgsl)

defaultdict(<function analyze_long.<locals>.<lambda> at 0x000002820EF7D3A0>, {'200': 3, 'unsolved': 4, '50': 10, '100': 6, '150': 4, '250': 2, '300': 1})

Ratio:  82
Gold:  The house Harry Potter is sorted into at Hogwarts is Gryffindor.
Res:  The house that Harry Potter is sorted into at Hogwarts is Thicket House.

Ratio:  65
Gold:  The four founders of Hogwarts are Godric Gryffindor, Salazar Slytherin, Rowena Ravenclaw, and Helga Hufflepuff.
Res:  The four founders of Hogwarts are Godric Glyndwr, Elara Brightshore, Alaric Nightshade, and Lyra Blackwood.

Ratio:  82
Gold:  The school Harry Potter attends is Hogwarts School of Witchcraft and Wizardry.
Res:  The school that Harry Potter attends is Hogwarts School of Wizardry and Witchcraft.

Ratio:  9
Gold:  The Muggle aunt and uncle that Harry must live with every summer are Vernon and Petunia Dursley.
Res:  The Muggle aunt and uncle are a recurring character in the Harry and Mia series. They are the parents of Mia's best friend, Emily